In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-094b68a5-e0df-e4cf-91ac-1a5d42480ae6)


#install pacakges

In [ ]:
%%capture
!python3 -m pip install -U pip
!python3 -m pip install -U setuptools wheel
!python3 -m pip install -U "mxnet_cu110<2.0.0"
!python3 -m pip install autogluon

In [ ]:
%%capture
!pip install -U transformers

# download data

In [ ]:
import os
if os.path.isdir('/content/Profiling-Hate-Speech-Spreaders-on-Twitter') is False:
    !git clone https://github.com/talhaanwarch/Profiling-Hate-Speech-Spreaders-on-Twitter.git

Cloning into 'Profiling-Hate-Speech-Spreaders-on-Twitter'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 50 (delta 10), reused 9 (delta 1), pack-reused 0
Unpacking objects: 100% (50/50), done.


In [ ]:
with open('pass.txt','r') as f:
  pas=f.read()

In [ ]:
import zipfile
if os.path.isdir('train') is False:
  train_path='/content/Profiling-Hate-Speech-Spreaders-on-Twitter/data/train.zip'
  with zipfile.ZipFile(train_path,"r") as zip_ref:
    zip_ref.extractall(".",pwd = bytes(pas,'utf-8'))
if os.path.isdir('test') is False:
  test_path='/content/Profiling-Hate-Speech-Spreaders-on-Twitter/data/test.zip'
  with zipfile.ZipFile(test_path,"r") as zip_ref:
    zip_ref.extractall(".",pwd = bytes(pas,'utf-8'))

# prepare data

## train data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df=pd.read_csv('train/es/truth.txt',sep=':::',header=None,engine='python')
df.columns=['id','label']

In [ ]:
df.label.value_counts()

1    100
0    100
Name: label, dtype: int64

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop = stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preprocessing(text):
    text=text.str.replace('\d+', '')
    text=text.str.replace('RT','')
    text=text.str.replace('#USER#','')
    text=text.str.replace('_','')
    text=text.str.replace('#URL#','')
    text= text.str.lower()
    text = text.str.replace('[^\w\s]','')
    #text = text.apply(lambda x : [lemmatizer.lemmatize(y) for y in w_tokenizer.tokenize(x)])
    #text = text.apply(lambda x: [item for item in x if item not in stop])
    #text = text.apply(lambda x : " ".join(x))
    return text

In [ ]:
df.id='train/es/'+df.id.astype(str)+'.xml'
df.head()

,id,label
0,train/es/1a698d93874644625d83a76b206c7aa3.xml,0
1,train/es/b7200193802bde3769f4daa371b5305b.xml,0
2,train/es/7ffb758291340303c2162fcee25e2780.xml,0
3,train/es/e309e9e2516824370310661c919dd5d8.xml,0
4,train/es/b523abc590889dfaf69a1c8470ef314d.xml,0


In [ ]:
len(df)

200

In [ ]:
import xml.etree.ElementTree as ET
def reader(df,ground=True):
  data=[]
  for x in df.iterrows():
      
      tree = ET.parse(x[1].id)
      root = tree.getroot()
      text=[x.text for x in root[0]]
      if ground:
        label=[x[1].label]*len(text)
        data.append(pd.DataFrame(zip(text,label),columns=['text','label']))
      else:
        data.append(pd.DataFrame(text,columns=['text']))
  return data

In [ ]:
data=reader(df)

In [ ]:
data[0].head()

,text,label
0,RT #USER#: Esperando para ver la captura de Ja...,0
1,#USER# #USER# Que bueno Romeo con la ayuda De ...,0
2,RT #USER#: Nuestras cuadrillas están restauran...,0
3,RT #USER#: Esta es la portada de #HASHTAG# de ...,0
4,RT #USER#: Lo más matado de la risa que vas a ...,0


In [ ]:

datax=data.copy()
for d in range(len(data)):
  datax[d].text=preprocessing(data[d].text)


In [ ]:
datax[0].head()

,text,label
0,esperando para ver la captura de jaime perla,0
1,que bueno romeo con la ayuda de dios serás u...,0
2,nuestras cuadrillas están restaurando parede...,0
3,esta es la portada de hashtag de este jueves...,0
4,lo más matado de la risa que vas a ver hoy,0


In [ ]:
len(datax[0])

200

In [ ]:
doc_text,doc_label,=[],[]
for i in range(len(datax)):
  doc_text.append(list(datax[i].text))
  doc_label.append(int(datax[i].mean()))

In [ ]:
len(doc_text),len(doc_label)

(200, 200)

## test data

In [ ]:
from glob import glob
test_path=pd.DataFrame(glob('test/es/*.xml'),columns=['id'])
test_path.head()

,id
0,test/es/335869e32becfc88c7d85eefa141540f.xml
1,test/es/69d556839af1b8b0756f1619a0178247.xml
2,test/es/3d6535ead17b2653c70059e7b7d5c2dd.xml
3,test/es/7b5c34c0d03cf69dd4a4c1d753d5a7e6.xml
4,test/es/e6fb1e54cd00d479017b9425977e28ab.xml


In [ ]:
test_data=reader(test_path,ground=False)

In [ ]:

test_datax=test_data.copy()
for d in range(len(test_datax)):
  test_datax[d].text=preprocessing(test_datax[d].text)


In [ ]:
test_doc=[]
for i in range(len(test_datax)):
  test_doc.append(list(test_datax[i].text))

In [ ]:
len(test_doc)

100

# generate embeddings

In [ ]:
from transformers import AutoTokenizer, AutoModel
from torch.nn.utils.rnn import pad_sequence
import torch
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased",)
model = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased",output_hidden_states=True)


Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dens

In [ ]:
device='cuda'
model=model.to(device)

In [ ]:
#get data from last hidden layer
def create_embeddings(doc_text,max_length=20,cat=False):
  doc_embeddings=[]
  for text in doc_text:
    inputs=tokenizer(text,max_length=max_length,padding=True,truncation=True)
    text_seq = torch.tensor(inputs['input_ids']).to(device)
    text_mask = torch.tensor(inputs['attention_mask']).to(device)
    model.eval()
    with torch.no_grad():
      outputs= model(text_seq,attention_mask=text_mask)
      hidden_states=outputs.hidden_states
      if cat:
        emb=torch.mean(torch.cat([hidden_states[i] for i in [-1,-2,-3,-4]], dim=-1),1)
      else:
        emb=torch.mean(hidden_states[11], 1)    
      doc_embeddings.append(emb)
  x=torch.mean(torch.stack(doc_embeddings),1)
  x=np.array(x.cpu())
  return x

## last layer

In [ ]:
%%time
train_last=create_embeddings(doc_text)
test_last=create_embeddings(test_doc)


CPU times: user 1min 4s, sys: 136 ms, total: 1min 4s
Wall time: 1min 3s


## last 4 layers

In [ ]:
%%time
train_last4=create_embeddings(doc_text,cat=True)
test_last4=create_embeddings(test_doc,cat=True)


CPU times: user 1min 11s, sys: 101 ms, total: 1min 11s
Wall time: 1min 10s


# AutoGluon

In [ ]:
def agdataframe(train,label,test):
  train_df=pd.concat([pd.DataFrame(train),pd.DataFrame(label)],axis=1)
  train_df.columns=['col_{}'.format(i) for i in range(train.shape[1])]+['Label']
  train_df = train_df.sample(frac=1).reset_index(drop=True)
  test_df=pd.DataFrame(test)
  test_df.columns=['col_{}'.format(i) for i in range(test.shape[1])]
  return train_df, test_df
  

In [ ]:
train_last_df,test_last_df=agdataframe(train_last,doc_label,test_last)
train_last4_df,test_last4_df=agdataframe(train_last4,doc_label,test_last4)

print(train_last_df.shape,test_last_df.shape,train_last4_df.shape,test_last4_df.shape)

(200, 769) (100, 768) (200, 3073) (100, 3072)


In [ ]:
from autogluon.tabular import  TabularPredictor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report,accuracy_score

def loop(data,test):
  acc=[]
  test_pred=[]
  if os.path.isdir('AutogluonModels'):
    !rm -rf AutogluonModels
  skf = StratifiedKFold(n_splits=5,shuffle=True)
  for train_index, val_index in skf.split(data.iloc[:,0:-1], data.iloc[:,-1]):
    train=data.iloc[train_index,:]
    val=data.iloc[val_index,:]
    clf=TabularPredictor(label='Label',verbosity=0).fit(train, presets='best_quality')
    res=clf.leaderboard(val,silent=True)
    res['score_test']=res['score_test'].round(3)*100
    pred = clf.predict(val,model=res.model[0])
    accuracy=accuracy_score(val.Label,pred)
    acc.append(accuracy)
    print('best model ',res.model[0],' accuracy is ',accuracy)
    test_pred.append(clf.predict(test,model=res.model[0]))
  print('average accuracy is',np.mean(acc))
  return test_pred,np.mean(acc)

In [ ]:
%%time
test_pred_last,test_acc_last=loop(train_last_df,test_last_df)

best model  LightGBMLarge_BAG_L1  accuracy is  0.85
best model  LightGBM_BAG_L1  accuracy is  0.85
best model  ExtraTreesGini_BAG_L1  accuracy is  0.85
best model  LightGBM_BAG_L1  accuracy is  0.85
best model  ExtraTreesGini_BAG_L1  accuracy is  0.875
average accuracy is 0.8550000000000001
CPU times: user 21min 34s, sys: 13.6 s, total: 21min 47s
Wall time: 13min 5s


In [ ]:
print(len(np.mean(test_pred_last,axis=0)),len(test_last_df)) #100

100 100


In [ ]:
%%time
test_pred_last4,test_acc_last4=loop(train_last4_df,test_last4_df)

best model  LightGBM_BAG_L1  accuracy is  0.825
best model  LightGBMLarge_BAG_L1  accuracy is  0.8
best model  RandomForestEntr_BAG_L1  accuracy is  0.875
best model  LightGBM_BAG_L1  accuracy is  0.85
best model  CatBoost_BAG_L1  accuracy is  0.9
average accuracy is 0.85
CPU times: user 1h 19min 37s, sys: 28.9 s, total: 1h 20min 6s
Wall time: 46min 22s


In [ ]:
def make_final(test_pred):
  ids=test_path.id.apply(lambda x:x.split('/')[-1].split('.')[0])
  pred=np.mean(test_pred,axis=0)
  lang=['es']*len(test_path)
  final=pd.DataFrame(zip(ids,lang,pred),columns=['author-id','lang','type'])
  return final

In [ ]:
test_last=make_final(test_pred_last)
test_last4=make_final(test_pred_last4)

In [ ]:
test_last.shape,test_last4.shape

((100, 3), (100, 3))

In [ ]:
test_last.to_csv('bertsapnish1_{}.csv'.format(test_acc_last),index=False)
test_last4.to_csv('bertbase2_{}.csv'.format(test_acc_last4),index=False)

In [ ]:
if os.path.isdir('output'):
  !rm -rf output
  !mkdir output
  !rm talha.zip
else:
  !mkdir output

In [ ]:
import pandas as pd
df=test_last.copy()
import pandas as pd
for row in df.index:
    xml = []
    xml.append('<author id="{}"'.format(df.loc[row,'author-id']))
    xml.append('lang="{}"'.format(df.loc[row,'lang']))
    xml.append('type="{}"'.format(df.loc[row,'type']))
    xml.append('/>')
    x='\n'.join(xml)
    myfile = open("output/{}.xml".format(df.loc[row,'author-id']), "w")
    myfile.write(x)
    myfile.close()
    

In [ ]:
%%capture
!zip -r talha.zip output